In [1]:
# Importing pandas
import pandas as pd


In [2]:
# Reading the csvs
!gdown "1SoKNC_Tbd7_ygUntnxErjFP57tJ1KlRR"
!gdown "1AA3N3quwh7emKGvhn5j_NPq1srrA3Mdf"

order_payments = pd.read_csv("olist_order_payments_dataset.csv")
orders_dataset = pd.read_csv("olist_orders_dataset.csv")

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\afeli\anaconda3\Scripts\gdown.exe\__main__.py", line 7, in <module>
  File "C:\Users\afeli\anaconda3\Lib\site-packages\gdown\cli.py", line 156, in main
    filename = download(
               ^^^^^^^^^
  File "C:\Users\afeli\anaconda3\Lib\site-packages\gdown\download.py", line 259, in download
    filename_from_url = m.groups()[0]
                        ^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'groups'
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\afeli\anaconda3\Scripts\gdown.exe\__main__.py", line 7, in <module>
  File "C:\Users\afeli\anaconda3\Lib\site-packages\gdown\cli.py", line 156, in main
    filename = download(
               ^^^^^^^^^
  File "C:\Users\afeli\anaconda3\Lib\site-packages\gdown\d

FileNotFoundError: [Errno 2] No such file or directory: 'olist_order_payments_dataset.csv'

We will first explore both datasets to see how they look like, showing the first five rows.

In [ ]:
# Checking the first five rows of the order_payments content
order_payments.head()

In [ ]:
# Checking the first five rows of the order_payments content
orders_dataset.head()

We will now merge these two datasets with inner join and using a common column on both, order_id. We will visualize how the datasets look like after the merge.

In [ ]:
# Merging both datasets
merged_orders = pd.merge(order_payments, orders_dataset, on= 'order_id', how= 'inner')
merged_orders.head()

There is a method called shape, that allows us to see the shape of the dataset. It will show a tuple of the number of rows, and the number of columns.

In [ ]:
# Showing the shape of merged_orders
merged_orders.shape

Now we will get a glimpse of some general elements of the dataset with the .info() method. It shows the dtype of the index and columns, the total amount of entries (rows) and columns, the total amount of columns per dtype, the non-null values amount per column, and memory usage.

In [ ]:
# Getting the database info
merged_orders.info()

There is another method, .describe(); that shows us the following values for numeric columns (float or numeric, in this case there are 3) unless we pass the 'all' to the parameter include: The total count of elements, the mean, the standard deviation, the minimum and maximum values and the 25th, 50th and 75th percentiles.

In [ ]:
# Getting the database description
merged_orders.describe()

We have seen that with the .info() method we get the total number of non-nulls, and if there is an inconsistency among those numbers, it means there are some null values. And the way to get the total amount of nulls is first applying the .isna() method that helps us detect them, and then we add all those occurrences with .sum().

In [ ]:
# Getting the total amount of null values per column
merged_orders.isna().sum()

There are 3 columns with several nulls. In this case, we are going to drop them with the .dropna() method. We can drop the rows with null values (axis = 0) or the columns (axis = 1).

Let's drop the rows that have NAs and check again if they were eliminated. There should be zero nulls.

In [ ]:
# Dropping the NAs and checking if they were eliminated
merged_orders.dropna(axis= 0, inplace= True)

merged_orders.isna().sum()

Our goal now is to create a new column with the average payment value per installment. But before that, if we pay attention to the payment_installments column information given by the .describe() method, we will see that the minimum value is 0, and math tells us that we can't divide a number by 0.

So what we will do right now, is to replace all those 0s by 1s on that column, using the .replace() method.

In [ ]:
# Replacing the 0s by 1s
merged_orders['payment_installments'] = merged_orders['payment_installments'].replace(0, 1)

How can we check if the 0s are gone? We can use describe again, but there is a neater way to see that, by using the .unique() method. This shows us the unique elements present.

So we are going to use that method on the payment_installments column to see them, and no 0 will be shown.

In [ ]:
# Showing the unique elements of the payment_isntallments column
merged_orders['payment_installments'].unique()

And there is a way to know how many unique elements that column has: the .nunique() method.

In [ ]:
# Showing the amount of unique elements of the payment_isntallments column
merged_orders['payment_installments'].nunique()

Now we can move on to create a new column that will have the payment value per installment. For this we can use the .apply() method. This is used to apply a function along an axis.

We will also show the new values on that newly created column.

In [ ]:
# Creating new column with the payment per installment and showing its content
merged_orders['payment_per_installment'] = merged_orders.apply(
    lambda row: row.payment_value / row.payment_installments, axis=1
    )

merged_orders['payment_per_installment']

Now let's check the unique elements of the payment_type column

In [ ]:
# Checking the unique elements of the payment_type column
merged_orders['payment_type'].unique()

There are four elements, and there are three in English and one in Portuguese. To be consistent, we will change the Portuguese word "boleto" by its English word "ticket".

For this we can use the .loc() method. With this we can access a group of rows and columns by label or labels or a boolean array.

After that, we will check the unique elements of that column again and "boleto" should have been replaced by "ticket".

In [ ]:
# Changing boleto in Portuguese for ticket in English and showing how the word was replaced
merged_orders.loc[(merged_orders.payment_type == 'boleto'),'payment_type']='ticket'
merged_orders['payment_type'].unique()

Antoher thing we can do is count the amount of values per element. For this we can use the .value_counts() method. Applied to the payment type it will show us the amount of occurrences of each of the four elements.

In [ ]:
# Showing the amount of occurrences per element for the payment_type column
merged_orders['payment_type'].value_counts()

Or we can see the amount of total payment values per element. For this we have the .groupby() method that allows us to group the dataframe by some criteria. We want to group them by the payment type.

In [ ]:
# Grouping the total payment values by payment_type
merged_orders.groupby(by='payment_type').sum()['payment_value']

We have used the .loc() method before. But there is a similar one called .iloc(). It lets us index what we want by the integer location.

Let's see the first row of the dataset using this method.

In [ ]:
# Checking the first row of the dataset with .iloc()
merged_orders.iloc[[0]]


We can also do the same thing but for the column. for that we will call all the rows (using ":") and the first column by its position, 0.

In [ ]:
# Checking the information of the first column using .loc()
merged_orders.iloc[:,[0]]

We can also index a specific value, for example the first element of the first column

In [ ]:
# Showing the first element of the first column with .iloc()
merged_orders.iloc[[0],[0]]

Different amount of rows or columns can be shown with this method as well.

In [ ]:
# Showing the first three rows of the dataset with .iloc()
merged_orders.iloc[0:3]